In [8]:
import os
os.chdir(os.path.join(os.getcwd(), '..','..', '..'))

In [9]:
from sklearn.naive_bayes import MultinomialNB
from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from sklearn.metrics import f1_score
import csv
import collections
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
import pandas as pd
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.filters import Filter
from weka.attribute_selection import ASEvaluation, AttributeSelection
from weka.classifiers import Classifier, Evaluation
from imblearn.over_sampling import SMOTE

jvm.start()

In [10]:
class NaiveBayes:
    def __init__(self, x_train, y_train, x_test, y_test, k):
        self.nb = MultinomialNB()
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.k = k
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)

    def feature_selection_SelectKBest(self):
        k_best = SelectKBest(chi2, k=self.k)
        k_best.fit(self.x_train, self.y_train)
        self.x_train = k_best.transform(self.x_train)
        self.x_test = k_best.transform(self.x_test)
        
    def feature_selection_ExtraTreesClassifier(self):
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(self.x_train, self.y_train)
        importances = clf.feature_importances_
        indices = np.argsort(importances)[::-1]
        self.x_train = self.x_train[:, indices[:self.k]]
        self.x_test = self.x_test[:, indices[:self.k]]

    def feature_selection_InfoGainAttributeEval(self, morbidity):
        loader = Loader(classname="weka.core.converters.ArffLoader")
        train_data = loader.load_file(f"./dataset/train/train_{morbidity}_tfidf.arff")
        train_data.class_is_last()

        # Initialize attribute selection
        eval = ASEvaluation(classname="weka.attributeSelection.InfoGainAttributeEval")
        search = AttributeSelection()
        search.evaluator = eval
        search.select_attributes(train_data)
        selected_attributes = search.selected_attributes
        filtered_attributes = np.delete(selected_attributes, [-1])
        # print("Selected attributes:", type(filtered_attributes), filtered_attributes.shape)

        # Apply selected attributes to the training and testing sets
        self.x_train = self.x_train[:, filtered_attributes]
        self.x_test = self.x_test[:, filtered_attributes]
        
    def train(self):
        self.nb.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.nb.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [11]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']
column_headings = ["Morbidity Class", "NB_Macro F1", "NB_Micro F1"]

In [12]:
with open("./results/tf-idf-features/performance_NB_AllFeatures.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    if len(collections.Counter(list(Y)).keys()) >=2:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled

        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 0)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)

    else:
        f1_macro = 1
        f1_micro = 1
        
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_NB_AllFeatures.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_NB_AllFeatures.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma
Macro F1 score: 0.6927668417822599 and Micro F1 Score 0.6941683168316831
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD
Macro F1 score: 0.7891579356931097 and Micro F1 Score 0.7907692307692308
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1 and Micro F1 Score 1
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression
Macro F1 score: 0.5836559110376284 and Micro F1 Score 0.5902173913043478
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes
Macro F1 score: 0.6838473040795018 and Micro F1 Score 0.686993670886076
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Gallstones
Macro F1 score: 0.5914327715626457 and Micro F1 Score 0.6017084061347312
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
GERD
Macro F1 score: 0.566411444594453 and Micro F1 Score 0.5711171171171171
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Gout
Macro F1 score: 0.6083269942359928 and Micro F1 Score 0.6138909634055265
(502, 600) (502,) Counter({1.

In [13]:
with open("./results/tf-idf-features/performance_NB_SelectKBest.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    if len(collections.Counter(list(Y)).keys()) >=2:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
            nb_obj.feature_selection_SelectKBest()
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)

    else:
        f1_macro = 1
        f1_micro = 1
        
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_NB_SelectKBest.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_NB_SelectKBest.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma
Macro F1 score: 0.727264737278525 and Micro F1 Score 0.731950495049505
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD
Macro F1 score: 0.8064768573489104 and Micro F1 Score 0.8076923076923077
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1 and Micro F1 Score 1
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression
Macro F1 score: 0.600399464530327 and Micro F1 Score 0.6065217391304347
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes
Macro F1 score: 0.7320454952397547 and Micro F1 Score 0.7349367088607595
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Gallstones
Macro F1 score: 0.6248551740904977 and Micro F1 Score 0.6303339157445155
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
GERD
Macro F1 score: 0.5663233767859444 and Micro F1 Score 0.5711171171171171
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Gout
Macro F1 score: 0.6050340123475016 and Micro F1 Score 0.6109316654219566
(502, 600) (502,) Counter({1.0

In [14]:
with open("./results/tf-idf-features/performance_NB_ExtraTreesClassifier.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    if len(collections.Counter(list(Y)).keys()) >=2:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
            nb_obj.feature_selection_ExtraTreesClassifier()
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)

    else:
        f1_macro = 1
        f1_micro = 1
        
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_NB_ExtraTreesClassifier.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_NB_ExtraTreesClassifier.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma
Macro F1 score: 0.7508668823474055 and Micro F1 Score 0.758029702970297
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD
Macro F1 score: 0.7927128387369575 and Micro F1 Score 0.793846153846154
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1 and Micro F1 Score 1
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression
Macro F1 score: 0.6335198326804271 and Micro F1 Score 0.6369565217391304
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes
Macro F1 score: 0.7942527633473985 and Micro F1 Score 0.7968512658227849
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Gallstones
Macro F1 score: 0.638365840279015 and Micro F1 Score 0.6402349058435255
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
GERD
Macro F1 score: 0.6030400777345213 and Micro F1 Score 0.6060720720720719
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Gout
Macro F1 score: 0.6589706523259271 and Micro F1 Score 0.6630227781926811
(502, 600) (502,) Counter({1.0

In [12]:
with open("./results/tf-idf-features/performance_NB_InfoGain.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()        
        
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)
        
        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
            nb_obj.feature_selection_InfoGainAttributeEval(morbidity)
            nb_obj.train()

            f1_macro, f1_micro = nb_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_NB_InfoGain.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_NB_InfoGain.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.4672986632039369 and Micro F1 Score 0.8776164549304294
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.4241422729474533 and Micro F1 Score 0.6112121212121211
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1 and Micro F1 Score 1
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Macro F1 score: 0.4408968801610552 and Micro F1 Score 0.790385739333723
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Macro F1 score: 0.409804351439744 and Micro F1 Score 0.6985902255639098
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Macro F1 score: 0.4599902140879871 and Micro F1 Score 0.8532485875706215
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
Macro F1 score: 0.4323776782842145 and Micro F1 Score 0.7638605442176871
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Macro F1 score: 0.4647957966912661 and Micro F1 Score 0.8691242937853108
(502, 600) (502,) Counter({1.0: 262, 0.0: 240})
Macro F1 score: 0.561839581623602